In [27]:
import h5py
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Reshape, Concatenate, Add
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import save_model, load_model

# from sklearn.model_selection import train_test_split

import os, sys, re, codecs

In [4]:
brand_f = h5py.File('X_dev_brand.h5py', 'r')
img_f = h5py.File('X_dev_img_feat.h5py', 'r')
maker_f = h5py.File('X_dev_maker.h5py', 'r')
model_f = h5py.File('X_dev_model.h5py', 'r')
product_f = h5py.File('X_dev_product.h5py', 'r')


OSError: Unable to open file (unable to open file: name = 'X_dev_brand.h5py', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [6]:
def load_data(chunk_num):
    # load chunk file with chunk_num
    chunk_file = 'origin/dev.chunk.0' + str(chunk_num)
    print("Load {} file...".format(chunk_file))
    
    with h5py.File(chunk_file, 'r') as hf:             
        
        pd_data = pd.DataFrame(columns=hf['dev'].keys())
        
        for col in pd_data.columns:
            if col == 'img_feat':
                sample_img = np.array(hf['dev']['img_feat'])
            else :
                pd_data[col] = hf['dev'][col]
             
        train_y = pd_data[['pid','bcateid', 'mcateid', 'scateid', 'dcateid']]
        train_x = pd_data.drop(columns=['bcateid', 'mcateid', 'scateid', 'dcateid', 'img_feat'])
        
        train_y = train_y.replace(-1, 0)
        
        # decode utf-8
        str_cols = ['brand', 'maker', 'product', 'model', 'pid', 'updttm']
        for col in str_cols:
            train_x[col] = train_x[col].str.decode('utf-8')
        
    return train_x, train_y, sample_img

In [7]:
train_x, train_y, sample_img = load_data(1)

Load origin/dev.chunk.01 file...


In [10]:
con_f = h5py.File('origin/X_dev_concate.h5py', 'r')

In [9]:
train_y['pid'] = train_y['pid'].str.decode('utf-8')

In [7]:
X_dev_product = np.array(product_f['dev'])
X_dev_maker = np.array(maker_f['dev'])
X_dev_brand = np.array(brand_f['dev'])
X_dev_model = np.array(model_f['dev'])
X_dev_image = np.array(img_f['dev'])
X_dev_con = np.array(con_f['dev'])

In [28]:
# print(X_dev_product.shape)
# print(X_dev_maker.shape)
# print(X_dev_brand.shape)
# print(X_dev_model.shape)
# print(X_dev_image.shape)
print(X_dev_con.shape)

(507783, 32)


In [29]:
model_path = 'dcate/models/'

In [30]:
d_model = load_model(model_path + 'd_model_con_unif_over_origin.h5')

In [31]:
d_prediction = d_model.predict(X_dev_con)
d_prediction = np.argmax(d_prediction, axis=1)

In [32]:
train_y['dcateid'] = d_prediction

In [33]:
train_y.head(50)

,pid,bcateid,mcateid,scateid,dcateid
0,H2829766805,0,0,0,120
1,H4269948000,0,0,0,15
2,H4692120833,0,0,0,116
3,O3630173399,0,0,0,4
4,G3523601788,0,0,0,66
5,J1691264922,0,0,0,6
6,S637556628,0,0,0,6
7,O4169340594,0,0,0,41
8,W2957690800,0,0,0,134
9,T4704824764,0,0,0,221


In [34]:
len(train_y)

507783

In [35]:
with open('d_con_l.tsv', 'w') as fout:
    for i in range(len(train_y)):
        ans = '{}\t{}\t{}\t{}\t{}\t'.format(train_y['pid'][i], train_y['bcateid'][i], train_y['mcateid'][i], train_y['scateid'][i], train_y['dcateid'][i])
        fout.write(ans)
        fout.write('\n')

In [31]:
dev_show = h5py.File('dev.chunk.01', 'r')

In [35]:
train_x.head()

,brand,maker,model,pid,price,product,updttm
0,기가바이트,GIGABYTE,"기가바이트 GB-BACE-3160 2GB, 500GB",H2829766805,267440,GIGABYTE 미니PC GB-BACE-3160 (램 4G+HDD 500GB) w,20180429194709
1,와코루,기타,와코루,H4269948000,-1,와코루 [WACOAL]와코루 튤레이스 홑겹 B컵브라 2칼라 (NB.SP)-DBR0156,20180413070637
2,,기타,기타,H4692120833,-1,카렉스 블랙스2 핸들커버(실버) 아반떼XD,20180415091715
3,뉴에라,뉴에라캡코리아,MLB 도트 프린트 뉴욕양키즈 티셔츠 11502825,O3630173399,61410,[뉴에라]MLB 도트 프린트 뉴욕 양키스 티셔츠 화이트(11502825),20180329085914
4,플러그피트니스,제조자:플러그피트니스,FROG,G3523601788,-1,[플러그피트니스] 네오플랜 삼각아령5kg/아령/여자아령/여성아령/팔운동/여성덤벨 [...,20180428044847


In [37]:
brand_tr = h5py.File('X_train_brand.h5py', 'r')
img_tr = h5py.File('X_train_img_feat.h5py', 'r')
maker_tr = h5py.File('X_train_maker.h5py', 'r')
model_tr = h5py.File('X_train_model.h5py', 'r')
product_tr = h5py.File('X_train_product.h5py', 'r')

In [97]:
np_brand = np.array(brand_tr['val'][:100])

In [98]:
np_img = np.array(img_tr['val'][:100])
np_maker = np.array(maker_tr['val'][:100])
np_model = np.array(model_tr['val'][:100])
np_product = np.array(product_tr['val'][:100])

In [106]:
b_train = b_model.predict([np_product,np_maker, np_brand, np_model,  np_img])
b_train = np.argmax(b_train, axis=1)

In [100]:
pd_b = pd.DataFrame(b_train)

In [107]:
b_train + 1

array([ 1, 13,  8, 13,  8, 15, 13, 15, 33,  8,  6,  5, 29, 23, 23, 21, 13,
       13,  8,  8, 27,  2,  4, 19, 21, 13, 19,  5, 13, 21, 16, 21, 13, 41,
        5,  2, 41,  1, 34, 12,  2, 29, 50,  2, 29, 19, 28, 23,  4, 35, 21,
        2,  4,  5, 21, 27,  6,  3, 49, 10,  2, 33, 39,  2, 21, 14, 21, 15,
        6, 27, 21,  8,  6,  8,  5,  8, 37, 17, 13, 27,  4, 17, 17,  8, 10,
       33, 36, 34, 32, 17, 22, 12, 22, 10, 24,  4, 16, 15, 21, 13])

In [56]:
y_bcate = h5py.File('y_train_bcateid.h5py', 'r')

In [104]:
np_y = y_bcate['val'][:100]
np_y = np.argmax(np_y, axis=1)

In [105]:
np_y

array([ 1, 13,  8, 13,  8, 15, 13, 15, 33,  8,  6,  5, 29, 23, 23, 21, 13,
       13,  8,  8, 27,  2,  4, 19, 21, 13, 19,  5, 13, 21, 16, 21, 13, 41,
        5,  2, 41,  1, 34, 12,  2, 29, 50,  2, 29, 19, 28, 23,  4, 35, 21,
        2,  4,  5, 21, 15,  6,  3, 49, 10,  2, 33, 39,  2, 21, 10, 21, 15,
        6, 27,  1,  8,  6,  8,  5,  8, 37, 17, 13, 27,  4, 17, 17,  8, 10,
       33, 36, 34, 32, 17, 22, 12, 22, 10, 24,  4, 16, 15, 21, 13])

In [ ]:
ans_y = pd.DataFrame(np_y)
ans_y[0].value_counts().sort_index()

In [1]:
!free

              total        used        free      shared  buff/cache   available
Mem:       61839832     4509648    42467312         904    14862872    56633920
Swap:             0           0           0


# lstm_model prediction

In [8]:
lstm_model = load_model('b_model_lstm.h5')

In [9]:
lstm_prediction = lstm_model.predict([X_tr_product,X_tr_model, X_tr_brand, X_tr_maker, sample_img])
print(lstm_prediction.shape)

(507783, 4205)


In [10]:
bcate = lstm_prediction[:, :57]
mcate = lstm_prediction[:, 57:609]
scate = lstm_prediction[:, 609:3800]
dcate = lstm_prediction[:, 3800:4205]

print("{} / {} / {} / {}".format(bcate.shape, mcate.shape, scate.shape, dcate.shape))

(507783, 57) / (507783, 552) / (507783, 3191) / (507783, 405)


In [11]:
bcate = np.argmax(bcate, axis=1) + 1
mcate = np.argmax(mcate, axis=1) + 1
scate = np.argmax(scate, axis=1)
dcate = np.argmax(dcate, axis=1)

In [12]:
train_y['bcateid'] = bcate
train_y['mcateid'] = mcate
train_y['scateid'] = scate
train_y['dcateid'] = dcate

In [13]:
train_y.head(50)

,pid,bcateid,mcateid,scateid,dcateid
0,H2829766805,28,94,0,0
1,H4269948000,9,43,0,0
2,H4692120833,28,94,0,0
3,O3630173399,9,43,0,0
4,G3523601788,14,13,0,0
5,J1691264922,3,3,38,0
6,S637556628,9,43,0,0
7,O4169340594,9,43,0,0
8,W2957690800,14,3,0,0
9,T4704824764,14,14,0,0


In [14]:
with open('test_lstm.tsv', 'w') as fout:
    for i in range(len(train_y)):
        ans = '{}\t{}\t{}\t{}\t{}\t'.format(train_y['pid'][i], train_y['bcateid'][i], train_y['mcateid'][i], train_y['scateid'][i], train_y['dcateid'][i])
        fout.write(ans)
        fout.write('\n')